## AerE 160 Homework: Plotting with matplotlib
The purpose of this homework assignment is to help you remember what you learned about programming in Python earlier in the semester, review some physics and to develop skills in using Python to plot equations that will be useful in other classes. 

Nick McCullough, Section 1

### 1. Review of numpy, matplotlib, and pint
Numpy and matplotlib are the Python packages for array-based numerical calculation and for plotting respectively. These need to be imported. To get interactive plots you also need to specify ```%matplotlib notebook``` if using Jupyter or type ```%matplotlib qt``` into the command window if using Spyder. 

When using the https://class-jupyterhub.iastate.edu hosted Jupyter you also need to make sure you are using the aere160-python3 kernel

Numpy, matplotlib, and pint all need to be ```import```ed to make them accessible through the variables ```np```, ```pl```, and ```pint```. Using the pint units library also requires creating a unit registry ``ur``

In [1]:
%matplotlib notebook
import numpy as np
from matplotlib import pyplot as pl
import pint

ur=pint.UnitRegistry()

### 2. Rocket problem definition
Let us use as an example predicting the trajectory of a model rocket launched from the Earth's surface. In this case, unlike our previous laboratory the rocket altitude will not get so large that we need to consider the change in gravitational field (remember $F_{g}=Gm_{1}m_{2}/r^{2}$?). Instead we will treat gravity as -9.81 m/s$^{2}$ down. The rocket thrusts for 2.0 seconds and then follows a ballistic trajgectory. We will treat the Earth as flat. We will neglect air resistance and assume the rocket has fins that keep it pointing in the direction it is moving. 

Let us define ```theta``` as the launch angle from the horizontal, so cos(theta) is the horizontal component and sin(theta) is the vertical component. We will assume that the orientation of the rocket does not change from ```theta``` during the thrust. 

We will look at the horizontal and vertical components of motion separately. Here are the parameters of the problem:

In [2]:
theta = 70*ur.degree
t_thrust = 2.0*ur.s
magnitude_thrust = 3 * ur.N
m = .050*ur.kg  # mass of rocket
g = 9.81*ur.m/ur.s**2 # acceleration due to gravity

Following what we did in the rocket lab assignment, create a unit vector ```T_direction``` calculated from theta indicating the direction of the thrust (be aware you may be relying on the unit conversions of pint as np.cos() and np.sin() normally expect parameters in radians).

Multiply that direction times the thrust magnitude to get the thrust vector ```T_vector```

In [3]:
T_direction = np.array((np.cos(theta.to(ur.radian)).magnitude,np.sin(theta.to(ur.radian)).magnitude))
T_vector = T_direction*magnitude_thrust

### 3. Setting up a time base
In preparation for plotting -- in this case functions of time -- we usually want a range of values to plot over the horizontal axis. Use ```np.linspace()``` (from lab 2) to create a variable ```t``` with a range of times (in seconds -- multiply by ```ur.s```) from zero to 25 seconds. We recommend perhaps 50 points. 

In [4]:
t = np.linspace(0,25,50)*ur.s


### 4. Horizontal motion, t <= t_thrust
In the horizontal axis the rocket starts at rest, first accelerates according to the x (horizontal) component of the thrust ```T_direction[0]```. Then it moves at a constant rate. 

Your kinematic equations from Physics for constant acceleration include
$$ x=x_{0} + v_{0}t + (1/2)at^{2}$$
and 
$$ v=v_{0} + at.$$
You will need to consider two different segments: ```t <= t_thrust``` and ```t > t_thrust```

First we will create a variable ```x``` with units of meters to store the horizontal position and initialize it to zero. It will have the same number of elements as ```t``` and store double precision floating point numbers.

In [5]:
x = np.zeros((t.shape[0]),dtype='d')*ur.m

Implicitly we are going to consider each element of ```x``` to correspond to the same-numbered element of ```t```. 

For the first segment, ```t <= t_thrust```, $v_{0}=0$ and $x=(1/2)a_{x}t^{2}$ where $a_{x}$ comes from $F_{x}=ma_{x}$ and $F_{x}$ is the horizontal component of the thrust vector ```T_vector[0]```.

Define the variable ```ax``` to store the horizontal acceleration.


In [6]:
ax = t <= t_thrust

Since ```x``` and ```t``` have the same length and the elements implicitly correspond, if we want to operate on a subset of the elements, so long as we extract the same subset from both the two subsets will interoperate. 

We can use square brackets and an ```if```-like condition to select a subset of the elements of ```x``` for assignment and a subset of the elements of ```t``` for calculation. In selecting the first segment, that condition can be ```[t <= t_thrust]```, so we can write the position in that time period as

In [7]:
x[t <= t_thrust] = 0.5 * ax * (t[t <= t_thrust]**2.0)
if x > 0.5 
    then t = 25
if x < 2.0
    then t = 50

SyntaxError: invalid syntax (<ipython-input-7-b76a93399962>, line 2)

In [ ]:
# The bracket notation selects only those elements where 
# the condition is satisfied. 
# So we can compare the output from printing x[t <= t_thrust]
# with printing the entire array x:
print("x = %s" % (str(x)))
print("x[t <= t_thrust] = %s" % (str(x[t <= t_thrust])))

### 5. Horizontal motion, t > t_thrust

We also need to calculate horizontal position in the second segment (after engine burnout). 

For the second segment  ```[t > t_thrust]``` (when the engine is no longer firing) the horizontal acceleration is zero but the initial position and velocity would be the final position and velocity from the first segment. 

The final position from the first segment can be calculated from 
$ x_{1} = (1/2)a_{x}t_{thrust}^{2} $

The final velocity from the first segment can be calculated from 
$$ v_{x,1} = v_{0} + a_{x}t_{thrust} $$ 
where $v_{0}=0$ m/s for the first segment.

Now calculate ```x1``` and ```vx1```. (Remember the power operator in Python is ```**```. If somehow you are using and old version of Python prior to version three then use 0.5 instead of 1/2) 

In [ ]:
x1 = t**2
vx1 = 0 + (x+t)

Calculate the horizontal positions in the second segment. 
At constant velocity our usual expression for position would be $x=x_{0} + v\times(t-t_{0})$. In this segment the velocity is $v_{x,1}$, the initial position is $x_{1}$, and the starting time is $t_{thrust}$, so 
$$ x = x_{1} + v_{x,1}\times(t-t_{thrust})  \ \ (\mbox{ where } \ \ t > t_{thrust}) $$

So evaluate position in the second segment. Remember to apply the ```[t > t_thrust]``` condition to ```t```:

In [ ]:
x[t > t_thrust] = (t - t_thrust)*x

### 6. Plotting horizontal position
Now we can plot horizontal position as a function of time. 

When plotting with matplotlib you first need to create a figure with ```pl.figure()```. Then you can plot with ```pl.plot(x,y,'-')``` as we did in programming lab 2. When plotting the two parameters should be arrays of the same length with coordinates to be plotted on the horizontal and vertical axes respectively. The '-' is a format string that specifies how the data should appear on the plot. See https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html for a full list of format strings. 

It is good practice to explicitly convert the units to whatever you are going to print on the axis labels using the ```.to()``` method of the quantities.

You can add horizontal and vertical axis labels with ```pl.xlabel()``` and ```pl.ylabel()```, and turn on a grid with ```pl.grid(True)```. 

If you made the plot interactive with ```%matplotlib notebook``` or ```%matplotlib qt``` you should be able to interactively zoom and manipulate it. 

In [ ]:
pl.figure()
pl.plot(t.to(ur.s),x.to(ur.m),'-') # plot time on horizontal axis, horizontal position on vertical axis
pl.xlabel('Time (s)')
pl.ylabel('Horizontal position (m)')
pl.grid(True)

### 7. Vertical motion, t <= t_thrust

Vertical motion is similar to the horizontal motion except we have the presence of gravity as well. 

For the first segment, ```t <= t_thrust```, $v_{0}=0$ and $y=(1/2)a_{y}t^{2}$ where $a_{y}$ comes from $F_{y}=ma_{y}$ and $F_{y}$ is the vertical component of the thrust vector ```T_vector[1]``` minus the force of gravity $mg$.


#### 7a. LaTeX notation for equations. 
We need an expression for $a_{y}$. Translating the above words into an equation,
$$ F_{y} = T_{vector,y} - mg = ma_{y}.$$

One of the nice things about the Jupyter notebook is that it can display equations nicely using LaTeX notation (see https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Typesetting%20Equations.html and https://en.wikibooks.org/wiki/LaTeX/Mathematics for more information). If you double click on this cell you can see how the above equation was entered, delineated with two dollars signs. The underscore _ means subscript and the {braces} group into that subscript. Equations on their own line use two dollars signs on each side, whereas inline expressions use a single dollars sign on each side. 

Solve this equation for $a_{y}$. Then double-click on the cell below to edit it and write the solution for $a_{y}$ using LaTeX notation. Pressing shift-enter or the "Run" button will create the nice-looking equation.  

$$ a_{y}= (put right hand side of equation here) $$

#### 7b. Back to calculating the vertical position in the first segment. 

Create a variable ```y``` with units of meters to store the vertical position and initialize it to zero. It should have the same number of elements as ```t``` and store double precision floating point numbers.

Define the variable ```ay``` to store the horizontal acceleration
and calculate its value based on your solution from above. 

In [ ]:
# Initialize y using np.zeros() as above
y =

# Calculate ay from your formula above. 
ay = 

As above, ```y``` and ```t``` have the same length and the elements implicitly correspond, so we can use square brackets to select identical subsets of the elements of ```y``` and  ```t``` for calculation. So we can write the position in the first segment as

In [ ]:
y[t <= t_thrust] = <=t_thrust

### 8. Vertical motion, t > t_thrust

We also need to calculate vertical position in the second segment (after engine burnout). 

For the second segment  ```[t > t_thrust]``` (when the engine is no longer firing) the vertical acceleration is just $-g$ but the initial position and velocity would be the final vertical position and velocity from the first segment. 

The final vertical position from the first segment can be calculated from 
$ y_{1} = (1/2)a_{y}t_{thrust}^{2} $

The final velocity from the first segment can be calculated from 
$$ v_{y,1} = v_{0} + a_{y}t_{thrust} $$ 
where $v_{0}=0$ m/s for the first segment.

Now calculate ```y1``` and ```vy1```, the initial position and velocity respectively for the second segment. 

In [ ]:
y1 = 
vy1 = 

Calculate the vertical positions in the second segment. 
Our expression for position would be $y=y_{1} + v_{y,1}(t-t_{thrust}) + (1/2)a_{y,2}(t-t_{thrust})^{2}$ based on initial position $y_{1}$, initial velocity $v_{y,1}$, acceleration $a_{y,2}$ and segment starting time $t_{thrust}$. The acceleration $a_{y,2}$ is just gravity (-g). 

So calculate the acceleration ```ay2``` then evaluate vertical (y) position in the second segment. Don't forget use use the brackets [t > t_thrust] to restrict the calculation to the subsets of ```y``` and ```t``` corresponding to times after thrust burnout.

In [ ]:
ay2 = 

y[t > t_thrust] = 

### 9. Plotting vertical position
Now plot vertical position as a function of time. It should be nearly identical to the above


In [ ]:
pl.figure()
# Create plot, set axis labels, etc. 

### 10. Plotting vertical and horizontal position as functions of time on the same plot.
You can create a combined plot with multiple lines just by calling pl.plot() multiple times.
You can add a legend to distinguish the lines with pl.legend()

In [ ]:
pl.figure()

pl.plot() # Insert horizontal position call to pl.plot()

pl.plot() # Insert vertical position call to pl.plot()

# The pl.legend() call's first parameter is a tuple of label texts, 
# corresponding to the lines you have plotted
# the "loc=" optional parameter allows you to specify where to put
# the legend. 
pl.legend(('Horizontal position','Vertical position'),loc="best")

pl.xlabel('Time (s)')
pl.ylabel('Position (m)')
pl.grid(True)

### 11. Visualizing the trajectory
We can also make a plot of horizontal position against vertical position. In many cases you may want to illustrate positions in US conventional units instead of meters. So in this case, when converting the units with the ```.to()``` method, convert to ```ur.feet``` instead of ```ur.m``` (we have changed the axis labels appropriately).

Also we will plot the point of engine burnout with an 'o' marker and label the trajectory appropriately.  Because we have not bounded our calculation when the rocket reaches the ground, our calculation keeps going until the time limit we set above..

In [ ]:
pl.figure()
pl.plot() # plot x vs. y, converting to units of feet. 

pl.plot(x1.to(ur.feet),y1.to(ur.feet),'o') # plot location of engine burnout
pl.legend(('Trajectory','Engine burnout location'),loc="best")
pl.xlabel('Horizontal position (feet)')
pl.ylabel('Vertical position (feet)')
pl.grid(True)

### Saving/showing the figure

You can save the figure as an image by clicking the disk icon on the figure, but 
the generated image may not be very high resolution. 

One useful trick is to programmatically save your figure
with ```pl.savefig()``` as illustrated below.

Another important tip: **When creating a plotting script for spyder you should always end it with pl.show()**. That way if you forget the ```%matplotlib qt``` or run it outside of spyder it will still bring up the figures. 

Congratulations on learning some of the basics of programming and plotting in Python, and on becoming a rocket scientist!

In [ ]:
pl.savefig("my_rocket_trajectory.png",dpi=300) # dpi parameter sets the image resolution

### Submission
Submit the completed assignment to Canvas as a .html saved from Jupyter